## Part 5.4: Neural Network
Now we are going to train a Feed-Forward Fully-Connected Neural Network to classify the samples. Different models are going to be trained in a manual grid search in order to find the combination of hyper-parameters that maximize the partial AUC.

In [25]:
# this function determines the partial area under the curve, up to FPR<=0.2
def partial_auc_at_fpr(y_true, y_pred_proba):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    valid_indices = fpr <= 0.2  # indices where FPR <= 0.2
    partial_fpr = fpr[valid_indices]
    partial_tpr = tpr[valid_indices]
    # check if there are at least 2 points for partial AUC calculation
    if len(partial_fpr) < 2:
        return 0  # return 0 if not enough points to calculate AUC

    partial_auc = auc(partial_fpr, partial_tpr)
    return partial_auc

def full_auc_scorer(y_true, y_pred_proba):
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    return auc(fpr, tpr)

# create scoring function (partial AUC) to implement a grid-search
partial_auc_scorer = make_scorer(partial_auc_at_fpr, greater_is_better=True)


In [ ]:

# redefine NN cycle
hidden_layer_sizes = [(18,), (36,), (54,), (36,18), (54,18), (54,36), (54,36,18)]
learning_rate_init = [ 0.1 , 0.01 , 0.001 ]

start_time = time.time()

auc_scorer = make_scorer(full_auc_scorer, greater_is_better=True, response_method='predict' ) 

parameters = []
fprs = []
tprs = []
aucs = []
partial_aucs = []
hls = []
lrs = []
K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=42) 

for size in hidden_layer_sizes:
    for lr in learning_rate_init:
        print(f"Training MLP of size {size} and lr {lr}...")
        lrs.append(lr)
        hls.append(size)
        partial_auc = 0
        auc_value = 0

        for i, (train_index, test_index) in enumerate(kf.split(X)):
            
            print(f"  Fold n. {i}")
            train_K=data_rescaled_and_indexed.iloc[train_idx]
            valid_K=data_rescaled_and_indexed.iloc[val_idx]
            X_train_K, Y_train_K = split_X_Y(train_K)
            X_valid_K, Y_valid_K = split_X_Y(valid_K)
            
            MLP = MLPClassifier(hidden_layer_sizes=size,
                                activation='relu', 
                                max_iter=600, 
                                alpha=1e-4, 
                                solver='sgd', 
                                tol=1e-4, 
                                learning_rate='adaptive',
                                learning_rate_init=lr,
                                verbose=0)
            
            MLP.fit(X_train_K,Y_train_K)
            parameters.append(MLP.coefs_)
            
            elapsed_time = time.time() - start_time
            start_time = time.time()
            
            y_pred_proba = MLP.predict_proba(X_valid_K)[:, 1]  
    
            fpr, tpr, _ = roc_curve(Y_valid_K, y_pred_proba)
            fprs.append(fpr)
            tprs.append(tpr)
            auc_value_k = auc(fpr, tpr)
            print("AUC (Test set):", auc_value)
            
            # search for fpr less than 0.2 (if not able fpr is set to zero)
            partial_fpr = fpr[fpr <= 0.2]
            if (len(partial_fpr) < 2 ):
                partial_auc = 0
            else:
                partial_tpr = tpr[:len(partial_fpr)]
                partial_auc_k = auc(partial_fpr, partial_tpr)
            partial_auc += partial_auc_k/K
            auc_value += auc_value_k/K                 
            partial_aucs.append(partial_auc)

        print(f"Time needed for training: {elapsed_time:.2f} seconds")
        print("partial AUC:",partial_auc," total AUC:",auc_value,'\n')


Training MLP of size (18,) and lr 0.1...
  Fold n. 0
AUC (Test set): 0
  Fold n. 1
AUC (Test set): 0.18541483179875334
  Fold n. 2
AUC (Test set): 0.3704760929685988
  Fold n. 3
AUC (Test set): 0.554921719559978
  Fold n. 4
AUC (Test set): 0.7403270725791972
Time needed for training: 2.07 seconds
partial AUC: 0.1367961056798852  total AUC: 0.9254265526742158 

Training MLP of size (18,) and lr 0.01...
  Fold n. 0
AUC (Test set): 0
  Fold n. 1
AUC (Test set): 0.18397272055859176
  Fold n. 2
AUC (Test set): 0.36870410804836484
  Fold n. 3
AUC (Test set): 0.5532743562860563
  Fold n. 4
AUC (Test set): 0.7382066209882661
Time needed for training: 2.35 seconds
partial AUC: 0.1345652506119433  total AUC: 0.92208935390269 

Training MLP of size (18,) and lr 0.001...
  Fold n. 0
AUC (Test set): 0
  Fold n. 1
AUC (Test set): 0.18200708903338628
  Fold n. 2
AUC (Test set): 0.3649364754677338
  Fold n. 3
AUC (Test set): 0.548092805758237
  Fold n. 4
AUC (Test set): 0.730967506926041
Time needed f

/home/samu/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


AUC (Test set): 0.5553794352409478
  Fold n. 4


/home/samu/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


AUC (Test set): 0.7381728984072308
Time needed for training: 6.26 seconds
partial AUC: 0.13528606318639502  total AUC: 0.9231201104156292 

Training MLP of size (36,) and lr 0.001...
  Fold n. 0


...

/home/samu/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


AUC (Test set): 0
  Fold n. 1


/home/samu/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/home/samu/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


AUC (Test set): 0.1820510802922502
  Fold n. 2
AUC (Test set): 0.35469313514582584
  Fold n. 3
AUC (Test set): 0.5216759332876205
  Fold n. 4


/home/samu/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Saving the best model's performance and plotting it

In [1]:
best_index_thresh = np.argmax(np.array(partial_aucs))
best_auc_thresh = aucs[best_index_thresh]
best_fpr_thresh = fprs[best_index_thresh]
best_tpr_thresh = tprs[best_index_thresh]
best_parameters = parameters[best_index_thresh]
best_size = hls[best_index_thresh]
best_lr = lrs[best_index_thresh]


best_NN = MLPClassifier(hidden_layer_sizes=best_size,
                                activation='relu', 
                                max_iter=600, 
                                alpha=1e-4, 
                                solver='sgd', 
                                tol=1e-4, 
                                learning_rate='adaptive',
                                learning_rate_init=best_lr,
                                verbose=0)
best_NN.fit(X_train,Y_train) # retrain the best
y_pred_proba_NN = best_NN.predict_proba(X_test)[:, 1]  

fpr_test, tpr_test, _ = roc_curve(Y_test, y_pred_proba_NN)
auc_test= auc(fpr_test,tpr_test)
partial_fpr_test = fpr_test[fpr_test <= 0.2]
if (len(partial_fpr_test)<2):
    partial_auc_test=0
else:
    partial_tpr_test = tpr_test[:len(partial_fpr_test)]
    partial_auc_test = auc(partial_fpr_test, partial_tpr_test)

fpr_NN = fpr_test
tpr_NN = tpr_test
Q_NN = get_best_Q(fpr_NN,tpr_NN)

print('best NN Q value:',Q_NN)

plt.grid()
plt.title('NN best ROC')
plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
plt.plot(fpr_test,tpr_test,label='NN ROC',ms=4,alpha=0.5,c='lightblue')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.legend()
plt.show()


NameError: name 'np' is not defined

Now we plot them all

In [ ]:
print('best NN Q value:',Q_NN.round(3))
print('best perceptron Q value:',Q_perceptron.round(3))
print('best SVM Q value:',Q_SVM.round(3))
print('best random forest Q value:',Q_rforest.round(3))

plt.grid()
plt.title('ROC comparison')
plt.plot([0, 1], [0, 1], linestyle='--', color='grey')
plt.plot(fpr_NN,tpr_NN,label='NN ROC',ms=2,alpha=0.4,c='#33ccff')
plt.plot(fpr_perceptron,tpr_perceptron,label='perceptron ROC',ms=2,alpha=0.4,c='red')
plt.plot(fpr_SVM,tpr_SVM,marker='o',label='SVM ROC',ms=2,alpha=0.4,c='orange')
plt.plot(fpr_rforest,tpr_rforest,label='random forest ROC',ms=2,alpha=0.4,c='#1aff1a')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.legend()
plt.show()